In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score,recall_score,confusion_matrix,f1_score,accuracy_score,classification_report,plot_confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from tensorflow.keras.layers import Embedding,  Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import pad_sequences
from keras.datasets import imdb 
# fix random seed for reproducibility 
np.random.seed(4)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#Handling text 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from wordcloud import WordCloud,STOPWORDS

In [ ]:
data_1 = pd.read_csv("/content/drive/MyDrive/ML + BD project data/data_1.csv",delimiter=',')
data_2 = pd.read_csv("/content/drive/MyDrive/ML + BD project data/data_2.csv",delimiter=',')

In [ ]:
data_1.head()

In [ ]:
data_1.shape

In [ ]:
data_2.head()

In [ ]:
data_2.shape

In [ ]:
data_2 = data_2.drop(columns=['Unnamed: 0'])
data_1 = data_1.drop(columns=['Unnamed: 0'])

In [ ]:
data_1.rename(columns = {'company_desc':'company_profile', 'logo':'has_company_logo', 'remote':'telecommuting' }, inplace = True)

In [ ]:
data_1.head()

In [ ]:
data = pd.concat([data_1,data_2], axis=0, ignore_index=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes.to_frame()

In [ ]:
categories=['telecommuting','has_company_logo','has_questions','employment_type','required_experience','required_education','industry','function','fraudulent']

In [ ]:
for i in categories:
     x=data[i].value_counts()
     print (x.to_frame())

In [ ]:
data.isna().sum()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(data.isna().transpose())
plt.xticks(rotation=45)

In [ ]:
plt.figure(figsize = (10,10))
corr = data.corr()
sns.heatmap(corr , mask=np.zeros_like(corr, dtype=np.bool) , cmap=sns.diverging_palette(-100,0,as_cmap=True) , square = True)

In [ ]:
#Removing undesired columns & nan 
data.function.fillna(data.department,inplace=True)
data.drop(columns=['job_id','salary_range','department'],inplace=True)

In [ ]:
#Now we need to handle missing values for text data and categorical data

text=['title','benefits','company_profile','location','description','requirements','fraudulent']

categ=['employment_type','required_experience','required_education','industry','function','telecommuting','has_company_logo','has_questions','fraudulent']

In [ ]:
#filling nan in categorical data
categ_cols=data[categ].fillna('None')
categ_cols

In [ ]:
#filling nan in text data
txt_cols=data[text].fillna(' ')
txt_cols

In [ ]:
categ_cols['country']=txt_cols['location'].apply(lambda x:x.split(',')[0])
countries=categ_cols['country'].value_counts().to_frame()
countries

In [ ]:
data['fraudulent'].value_counts().to_frame()

In [ ]:
colors = sns.color_palette('Set2')[0:10]
labels=['Real Job','Fake Job']
plt.figure(figsize=(10,8))
plt.title('Huge Imbalance in Target Values',size=20)
sns.set_style('whitegrid')
plt.pie(data['fraudulent'].value_counts(),labels=labels,colors=colors,autopct='%.0f%%')

In [ ]:
fakejobs=categ_cols[categ_cols['fraudulent']==1]
realjobs=categ_cols[categ_cols['fraudulent']==0]

In [ ]:
sns.set_style('darkgrid')
fig,axes=plt.subplots(5,2,figsize=(30,50))
sns.countplot(fakejobs['country'],palette='Set2',order = fakejobs['country'].value_counts()[:5].index,ax=axes[0,0])
sns.countplot(realjobs['country'],palette='Set2',order = realjobs['country'].value_counts()[:5].index,ax=axes[0,1])
axes[0,0].set_title('Highest 5 Countries of Fake Jobs',fontsize=15)
axes[0,1].set_title('Highest 5 Countries of Real Jobs',fontsize=15)
axes[0,0].set_xlabel('Country')
axes[0,1].set_xlabel('Country')

sns.countplot(fakejobs['employment_type'],palette='Set2',order = fakejobs['employment_type'].value_counts()[:5].index,ax=axes[1,0])
sns.countplot(realjobs['employment_type'],palette='Set2',order = realjobs['employment_type'].value_counts()[:5].index,ax=axes[1,1])
axes[1,0].set_title('Fake Jobs Most Employment Types',fontsize=15)
axes[1,1].set_title('Real Jobs Most Employment Types',fontsize=15)
axes[1,0].set_xlabel('Employment Type')
axes[1,1].set_xlabel('Employment Type')


sns.countplot(fakejobs['required_experience'],palette='Set2',order = fakejobs['required_experience'].value_counts()[1:5].index,ax=axes[2,0])
sns.countplot(realjobs['required_experience'],palette='Set2',order = realjobs['required_experience'].value_counts()[1:5].index,ax=axes[2,1])
axes[2,0].set_title('Fake Jobs Most Required Experience',fontsize=15)
axes[2,1].set_title('Real Jobs Most Required Experience',fontsize=15)
axes[2,0].set_xlabel('Required Experience')
axes[2,1].set_xlabel('Required Experience')

sns.countplot(fakejobs['industry'],palette='Set2',order = fakejobs['industry'].value_counts()[1:5].index,ax=axes[3,0])
sns.countplot(realjobs['industry'],palette='Set2',order = realjobs['industry'].value_counts()[1:5].index,ax=axes[3,1])
axes[3,0].set_title('Fake Jobs Most Industries',fontsize=15)
axes[3,1].set_title('Real Jobs Most Industries',fontsize=15)
axes[3,0].set_xlabel('Industryy')
axes[3,1].set_xlabel('Industry')

sns.countplot(fakejobs['function'],palette='Set2',order = fakejobs['function'].value_counts()[1:5].index,ax=axes[4,0])
sns.countplot(realjobs['function'],palette='Set2',order = realjobs['function'].value_counts()[1:5].index,ax=axes[4,1])
axes[4,0].set_title('Fake Jobs Most Functions',fontsize=15)
axes[4,1].set_title('Real Jobs Most Functions',fontsize=15)
axes[4,0].set_xlabel('Function')
axes[4,1].set_xlabel('Function')

# AU --> Austrilia
# GB --> United Kingdom
# GR --> Greece
# CA --> Canada
# MY --> Malaysia

In [ ]:
fig,axes=plt.subplots(1,3,figsize=(21,7))
sns.countplot(x='telecommuting', data=categ_cols, hue='fraudulent', palette='Set2',ax=axes[0]) 
sns.countplot(x='has_company_logo', data=categ_cols, hue='fraudulent', palette='Set2',ax=axes[1]) 
sns.countplot(x='has_questions', data=categ_cols, hue='fraudulent', palette='Set2',ax=axes[2]) 

axes[0].set_title('Telecommuting',fontsize=12)
axes[1].set_title('Has Company Logo?',fontsize=12)
axes[2].set_title('Has Questions?',fontsize=12)

axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[2].set_xlabel('')

In [ ]:
exp_list=['Executive','Associate','Director','Mid-Senior level']
exp_data=categ_cols.loc[categ_cols['required_experience'].isin(exp_list)]

In [ ]:
exp_data_indus=exp_data.groupby('industry')['required_experience'].value_counts(ascending=False).to_frame()
exp_data_indus.head(30)

In [ ]:
exp_data_fun=exp_data.groupby('function')['required_experience'].value_counts(ascending=False).to_frame()
exp_data_fun.head(30)

In [ ]:
categ_cols.groupby('country')['employment_type'].value_counts(ascending=False).to_frame().head(50)

In [ ]:
categ_cols.groupby('industry')['required_education'].value_counts(ascending=False).to_frame().head(50)

In [ ]:
# As we used Country in location column as a category data
cols_to_remove=['location']
txt_cols.drop(columns=cols_to_remove,inplace=True) 

In [ ]:
stemmer=PorterStemmer()
stop=set(stopwords.words('english'))

In [ ]:
def column_clean(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z\s*]','',text)
    text=text.split()
    text=[stemmer.stem(word) for word in text if word not in set(stopwords.words('english'))]
    return (text)

In [ ]:
# Splitting Text Data to Fraud or Not Fraud
txt_fraud=txt_cols[txt_cols['fraudulent']==1]
txt_not_fraud=txt_cols[txt_cols['fraudulent']==0]

In [ ]:
txt_fraud['title']=txt_fraud['title'].apply(column_clean)
txt_not_fraud['title']=txt_not_fraud['title'].apply(column_clean)

In [ ]:
def combine(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

fraud_titles = txt_fraud['title'].apply(combine)
not_fraud_titles = txt_not_fraud['title'].apply(combine)

In [ ]:
fraud_titles

In [ ]:
# To List

fraud_titles_corpus=fraud_titles.values.tolist()
not_fraud_titles_corpus=not_fraud_titles.values.tolist()

In [ ]:
# Corpus for WordCloud
fraud_titles_corpus=' '.join(fraud_titles_corpus)
not_fraud_titles_corpus=' '.join(not_fraud_titles_corpus)

In [ ]:
plt.figure(figsize=(15,10))
wordcloudfraud=WordCloud(width = 1900 , height = 1200 , background_color='white').generate(fraud_titles_corpus)
plt.axis('off')
plt.title('Most Common titles words in Fake jobs',size=20)
plt.imshow(wordcloudfraud)

In [ ]:
plt.figure(figsize=(15,10))
wordcloudnotfraud=WordCloud(width = 1900 , height = 1200,background_color='white').generate(not_fraud_titles_corpus)
plt.axis('off')
plt.title('Most Common titles words in Real jobs',size=20)
plt.imshow(wordcloudnotfraud)

In [ ]:
fraud_title_length=txt_cols[txt_cols["fraudulent"]==1]['title'].str.len()
not_fraud_title_length=txt_cols[txt_cols["fraudulent"]==0]['title'].str.len()

fig,axes=plt.subplots(1,2,figsize=(21,7))
sns.kdeplot(fraud_title_length,ax=axes[0],color='black') 
sns.kdeplot(not_fraud_title_length,ax=axes[1],color='black') 

axes[0].set_title('Fake Title Lengths',fontsize=12)
axes[1].set_title('Not Fake Titles Lengths',fontsize=12)

In [ ]:
txt_fraud['benefits']=txt_fraud['benefits'].apply(column_clean)
txt_not_fraud['benefits']=txt_not_fraud['benefits'].apply(column_clean)

In [ ]:
fraud_benefits = txt_fraud['benefits'].apply(combine)
not_fraud_benefits = txt_not_fraud['benefits'].apply(combine)

In [ ]:
# To List

fraud_benefits_corpus=fraud_benefits.values.tolist()
not_fraud_benefits_corpus=not_fraud_benefits.values.tolist()

In [ ]:
# Corpus for WordCloud
fraud_benefits_corpus=' '.join(fraud_benefits_corpus)
not_fraud_benefits_corpus=' '.join(not_fraud_benefits_corpus)

In [ ]:
plt.figure(figsize=(15,10))
wordcloud_fraud_ben=WordCloud(width = 1900 , height = 1200,background_color='white').generate(fraud_benefits_corpus)
plt.axis('off')
plt.title('Most Common Words in Benefits in Fake jobs',size=20)
plt.imshow(wordcloud_fraud_ben)

In [ ]:
plt.figure(figsize=(15,10))
wordcloud_notfraudben=WordCloud(width = 1900 , height = 1200,background_color='white').generate(not_fraud_benefits_corpus)
plt.axis('off')
plt.title('Most Common Words in Benefits in Real jobs',size=20)
plt.imshow(wordcloud_notfraudben)

In [ ]:
fraud_benefits_length=txt_cols[txt_cols["fraudulent"]==1]['benefits'].str.len()
not_fraud_benefits_length=txt_cols[txt_cols["fraudulent"]==0]['benefits'].str.len()

fig,axes=plt.subplots(1,2,figsize=(21,7))
sns.kdeplot(fraud_benefits_length,ax=axes[0],color='black') 
sns.kdeplot(not_fraud_benefits_length,ax=axes[1],color='black') 

axes[0].set_title('Fake Benefits Lengths',fontsize=12)
axes[1].set_title('Real Benefits Lengths',fontsize=12)

In [ ]:
txt_fraud['company_profile']=txt_fraud['company_profile'].apply(column_clean)
txt_not_fraud['company_profile']=txt_not_fraud['company_profile'].apply(column_clean)

In [ ]:
fraud_company = txt_fraud['company_profile'].apply(combine)
not_fraud_company = txt_not_fraud['company_profile'].apply(combine)

In [ ]:
# To List

fraud_company_corpus=fraud_company.values.tolist()
not_fraud_company_corpus=not_fraud_company.values.tolist()

In [ ]:
# Corpus for WordCloud
fraud_company_corpus=' '.join(fraud_company_corpus)
not_fraud_company_corpus=' '.join(not_fraud_company_corpus)

In [ ]:
plt.figure(figsize=(15,10))
wordcloud_fraud_company=WordCloud(width = 1900 , height = 1200,background_color='white').generate(fraud_company_corpus)
plt.axis('off')
plt.title('Most Common Words in Company Profile in Fake jobs',size=20)
plt.imshow(wordcloud_fraud_company)

In [ ]:
plt.figure(figsize=(15,10))
wordcloud_not_fraud_company=WordCloud(width = 1900 , height = 1200,background_color='white').generate(not_fraud_company_corpus)
plt.axis('off')
plt.title('Most Common Words in Company Profile in Real jobs',size=20)
plt.imshow(wordcloud_not_fraud_company)

In [ ]:
fraud_company_length=txt_cols[txt_cols["fraudulent"]==1]['company_profile'].str.len()
not_fraud_company_length=txt_cols[txt_cols["fraudulent"]==0]['company_profile'].str.len()

fig,axes=plt.subplots(1,2,figsize=(21,7))
sns.kdeplot(fraud_company_length,ax=axes[0],color='black') 
sns.kdeplot(not_fraud_company_length,ax=axes[1],color='black') 

axes[0].set_title('Fake Company Profile Lengths',fontsize=12)
axes[1].set_title('Real Company Profile Lengths?',fontsize=12)

In [ ]:
txt_fraud['requirements']=txt_fraud['requirements'].apply(column_clean)
txt_not_fraud['requirements']=txt_not_fraud['requirements'].apply(column_clean)

In [ ]:
fraud_requirements = txt_fraud['requirements'].apply(combine)
not_fraud_requirements = txt_not_fraud['requirements'].apply(combine)

In [ ]:
# To List

fraud_requirements_corpus=fraud_requirements.values.tolist()
not_fraud_requirements_corpus=not_fraud_requirements.values.tolist()

In [ ]:
# Corpus for WordCloud
fraud_requirements_corpus=' '.join(fraud_requirements_corpus)
not_fraud_requirements_corpus=' '.join(not_fraud_requirements_corpus)

In [ ]:
plt.figure(figsize=(15,10))
wordcloud_fraud_requirements=WordCloud(width = 1900 , height = 1200,background_color='white').generate(fraud_requirements_corpus)
plt.axis('off')
plt.title('Most Common Words in Requirements in Fake jobs',size=20)
plt.imshow(wordcloud_fraud_requirements)

In [ ]:
plt.figure(figsize=(15,10))
wordcloud_notfraud_requirements=WordCloud(width = 1900 , height = 1200,background_color='white').generate(not_fraud_requirements_corpus)
plt.axis('off')
plt.title('Most Common Words in Requirements in Real jobs',size=20)
plt.imshow(wordcloud_notfraud_requirements)

In [ ]:
data=txt_cols.join(categ_cols.drop(columns='fraudulent'))

In [ ]:
data['text']=data['title'] +' '+data['benefits']+' '+data['company_profile']+' '+data['description']+' '+data['requirements']
colm=['title','benefits','company_profile','description','requirements']
data.drop(columns=colm,inplace=True)

In [ ]:
data['text']=data['text'].apply(column_clean)

In [ ]:
data['text']=data['text'].apply(combine)

In [ ]:
plt.figure(figsize = (15,10)) 
wordcloud_real = WordCloud(width = 1600 , height = 800 , max_words = 500,background_color='white').generate(" ".join(data[data['fraudulent'] == 0]['text']))
plt.axis('off')
plt.title('Most Common Words in Real jobs',size=20)
plt.imshow(wordcloud_real)

In [ ]:
plt.figure(figsize = (15,10)) 
wordcloud_fake = WordCloud(width = 1600 , height = 800 , max_words = 500,background_color='white').generate(" ".join(data[data['fraudulent'] == 1]['text']))
plt.axis('off')
plt.title('Most Common Words in Fake jobs',size=20)
plt.imshow(wordcloud_fake)

In [ ]:
x=data.drop(columns='fraudulent')
y=data['fraudulent']

In [ ]:
# Applying Count Vectorizer
count_vec = CountVectorizer(max_features=5000)
vec = count_vec.fit_transform(data['text'])

In [ ]:
text=pd.DataFrame(vec.toarray(),columns=count_vec.get_feature_names_out())

In [ ]:
labels=pd.get_dummies(x.drop(columns='text'))
result = pd.concat([labels, text], axis=1)

In [ ]:
# Splitting data to train and test
x_train,x_test,y_train,y_test=train_test_split(result,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
# Naive Bayes
nb=MultinomialNB()
nb.fit(x_train,y_train)
nb_y_pred=nb.predict(x_test)

In [ ]:
print('        Training Classification report for Naive Bayes \n',classification_report(y_train,nb.predict(x_train)))
print('        Testing Classification Report for Naive Bayes \n',classification_report(y_test,nb_y_pred))

print('        Confusion Matrix for Training Naive Bayes \n',plot_confusion_matrix(nb,x_train,y_train))
print('        Confusion Matrix for Testing Naive Bayes \n',plot_confusion_matrix(nb,x_test,y_test))

In [ ]:
# Logistic Regression
lr=LogisticRegression()
lr.fit(x_train,y_train)
lr_y_pred=lr.predict(x_test)

In [ ]:
print('        Training Classification report for Logistic Regression \n',classification_report(y_train,lr.predict(x_train)))
print('        Testing Classification Report for Logistion Regression \n',classification_report(y_test,lr_y_pred))

print('        Confusion Matrix for Training Logistic Regression \n',plot_confusion_matrix(lr,x_train,y_train))
print('        Confusion Matrix for Testing Logistic Regression \n',plot_confusion_matrix(lr,x_test,y_test))

In [ ]:
# KNN
knn= KNeighborsClassifier()
knn.fit(x_train,y_train)
knn_y_pred=knn.predict(x_test)

In [ ]:
print('        Training Classification report for KNN \n',classification_report(y_train,lr.predict(x_train)))
print('        Testing Classification Report for KNN \n',classification_report(y_test,knn_y_pred))

print('        Confusion Matrix for Training KNN \n',plot_confusion_matrix(knn,x_train,y_train))
print('        Confusion Matrix for Testing KNN \n',plot_confusion_matrix(knn,x_test,y_test))

In [ ]:
# DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
dt_y_pred=dt.predict(x_test)

In [ ]:
print('        Training Classification report for Decision Tree Classifier \n',classification_report(y_train,dt.predict(x_train)))
print('        Testing Classification Report for Decision Tree Classifier \n',classification_report(y_test,dt_y_pred))
print('        Confusion Matrix for Training Decision Tree Classifier \n',plot_confusion_matrix(dt,x_train,y_train))
print('        Confusion Matrix for Testing Decision Tree Classifier\n',plot_confusion_matrix(dt,x_test,y_test))

In [ ]:
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
rf_y_pred=rf.predict(x_test)

In [ ]:
print('        Training Classification report for Random Forest Classifier \n',classification_report(y_train,rf.predict(x_train)))
print('        Testing Classification Report for Random Forest Classifier \n',classification_report(y_test,rf_y_pred))


print('        Confusion Matrix for Training Random Forest Classifier \n',plot_confusion_matrix(rf,x_train,y_train))
print('        Confusion Matrix for Testing Random Foresr Classifier\n',plot_confusion_matrix(rf,x_test,y_test))

In [ ]:
x_train.head()

In [ ]:
xgb=xgb.XGBClassifier(max_depth=6, learning_rate=0.1,silent=False, objective='binary:logistic')
xgb.fit(x_train,y_train)
xgb_y_pred=xgb.predict(x_test)

In [ ]:
print('        Training Classification report for XGBoost Classifier \n',classification_report(y_train,xgb.predict(x_train)))
print('        Testing Classification Report for XGBoost Classifier \n',classification_report(y_test,xgb_y_pred))


print('        Confusion Matrix for Training XGBoost Classifier \n',plot_confusion_matrix(xgb,x_train,y_train))
print('        Confusion Matrix for Testing XGBoost Classifier\n',plot_confusion_matrix(xgb,x_test,y_test))

In [ ]:
ada=AdaBoostClassifier()
ada.fit(x_train,y_train)
ada_y_pred=ada.predict(x_test)

In [ ]:
print('        Training Classification report for AdaBoost Classifier \n',classification_report(y_train,ada.predict(x_train)))
print('        Testing Classification Report for AdaBoost Classifier \n',classification_report(y_test,ada_y_pred))


print('        Confusion Matrix for Training AdaBoost Classifier \n',plot_confusion_matrix(ada,x_train,y_train))
print('        Confusion Matrix for Testing AdaBoost Classifier\n',plot_confusion_matrix(ada,x_test,y_test))

In [ ]:
data.head()

In [ ]:
max_features = 10000
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer

# create the tokenizer
t = Tokenizer(num_words = max_features)
# fit the tokenizer on the documents
t.fit_on_texts(data['text'].values)

In [ ]:
data['word count'] = [len(i.split(' ')) for i in data['text']]
sent_length = data['word count'].max()

In [ ]:
encoded_docs = t.texts_to_sequences(data['text'].values)
embedded_docs=pad_sequences(encoded_docs,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
y = data['fraudulent'].values
y

In [ ]:
y = y.reshape(-1,1)
y.shape

In [ ]:
X = np.array(embedded_docs)
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 101)

In [ ]:
print("X_train shape: ",X_train.shape)
print("X_test shape : ",X_test.shape )
print("y_train shape: ",y_train.shape)
print("y_test shape : ",y_test.shape)

In [ ]:
def eval_metrics(actual, prediction):
    print("Accuracy Score: {}".format(accuracy_score(actual, prediction)))
    print("Recall Score: {}".format(recall_score(actual, prediction)))
    print("f1 Score: {}".format(f1_score(actual, prediction)))

In [ ]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(max_features,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(20)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
model1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

hist = model1.fit(X_train, y_train, epochs = 5, batch_size = 64, validation_data=(X_test,y_test))

In [ ]:
plt.plot(hist.history['val_loss'], color='b', label="validation loss")
plt.plot(hist.history['loss'], color='red', label="loss")
plt.title("Model Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['val_accuracy'], color='b', label="validation accuracy")
plt.plot(hist.history['accuracy'], color='red', label="accuracy")
plt.title("Model Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
y_pred = model1.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
y_train_pred = model1.predict(X_train)
y_train_pred = (y_train_pred > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
print('        Training Classification report for Bidirectional LSTM \n',classification_report(y_train,y_train_pred))
print('        Testing Classification Report for Bidirectional LSTM \n',classification_report(y_test,y_pred))



print('        Confusion Matrix for Testing Bidirectional LSTM \n')
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()


In [ ]:
print('        Confusion Matrix for Training Bidirectional LSTMr\n')
cm = confusion_matrix(y_train, y_train_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(max_features,embedding_vector_features,input_length=sent_length))
model2.add(LSTM(25))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())


In [ ]:
model2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
from keras.callbacks import EarlyStopping, ModelCheckpoint

hist = model2.fit(X_train, y_train, epochs = 5, batch_size = 64, validation_data=(X_test,y_test))

In [ ]:
plt.plot(hist.history['val_loss'], color='b', label="validation loss")
plt.plot(hist.history['loss'], color='red', label="loss")
plt.title("Model Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['val_accuracy'], color='b', label="validation accuracy")
plt.plot(hist.history['accuracy'], color='red', label="accuracy")
plt.title("Model Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
y_pred = model2.predict(X_test)
y_pred = (y_pred > 0.5)
y_train_pred = model2.predict(X_train)
y_train_pred = (y_train_pred > 0.5)

In [ ]:
print('        Training Classification report for LSTM \n',classification_report(y_train,y_train_pred))
print('        Testing Classification Report for LSTM \n',classification_report(y_test,y_pred))



print('        Confusion Matrix for Testing LSTM \n')
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
print('        Confusion Matrix for Training LSTMr\n')
cm = confusion_matrix(y_train, y_train_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(max_features,embedding_vector_features,input_length=sent_length))
model3.add(Dense(256))
model3.add(Bidirectional(LSTM(25)))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())

In [ ]:
model3.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
from keras.callbacks import EarlyStopping, ModelCheckpoint

hist = model3.fit(X_train, y_train, epochs = 5, batch_size = 64, validation_data=(X_test,y_test))

In [ ]:
plt.plot(hist.history['val_loss'], color='b', label="validation loss")
plt.plot(hist.history['loss'], color='red', label="loss")
plt.title("Model Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['val_accuracy'], color='b', label="validation accuracy")
plt.plot(hist.history['accuracy'], color='red', label="accuracy")
plt.title("Model Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
y_pred = model1.predict(X_test)
y_pred = (y_pred > 0.5)
y_train_pred = model1.predict(X_train)
y_train_pred = (y_train_pred > 0.5)

In [ ]:
print('        Training Classification report for Bidirectional LSTM \n',classification_report(y_train,y_train_pred))
print('        Testing Classification Report for Bidirectional LSTM \n',classification_report(y_test,y_pred))



print('        Confusion Matrix for Testing Bidirectional LSTM \n')
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
print('        Confusion Matrix for Training Bidirectional LSTMr\n')
cm = confusion_matrix(y_train, y_train_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
from keras import backend as K
from keras import optimizers


def create_base_model(activation = 'tanh',  loss='binary_crossentropy', optimizer = "Adam", lr = 0.01):
  embedding_vector_features=40
  model=Sequential()
  model.add(Embedding(max_features,40,input_length=sent_length))
  model.add(LSTM(25, activation = activation))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss=loss,optimizer=optimizer,metrics=['accuracy'])
  K.set_value(model.optimizer.learning_rate, lr)
  return model

def k_fold_test(X, y, epochs = 5, batch_size = 64, activation = 'tanh', loss='binary_crossentropy', optimizer = "Adam", lr = 0.01):
  from sklearn.model_selection import StratifiedKFold,KFold
  k_fold_index = StratifiedKFold(n_splits = 5,shuffle=True,random_state=42)
  early_stopping_monitor = EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    verbose=1,
  )
  val_accuracy = 0
  for train_index, test_index in k_fold_index.split(X,y):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_base_model(activation, loss, optimizer)
    hist = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data=(x_test,y_test), callbacks=[early_stopping_monitor])

    val_accuracy += max(hist.history['val_accuracy'])

  val_accuracy = val_accuracy/5

  return val_accuracy

In [ ]:
# from keras import optimizers

# epochs = [5,10]
# activations = ['tanh']
# optimizers_arr = ["Adam", "SGD"]
# lrs = [0.1, 0.01, 0.001]


In [ ]:
# results = []
# for e in epochs:
#   for a in activations:
#     for o in optimizers_arr:
#       for l in lrs:
#         val =  k_fold_test(X, y, epochs = e, batch_size = 64, activation = a, loss='binary_crossentropy', optimizer = o, lr = l)
#         x = 'Epochs: ' + str(e) + ' Activation function: ' + str(a) + ' Optimizer: ' + str(o) + ' LR: ' + str(l) + ' Cross-validational test accuracy: ' + str(val)
#         results.append(x) 
#         print(x)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(max_features,40,input_length=sent_length))
model.add(LSTM(25, activation = 'tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
K.set_value(model.optimizer.learning_rate, 0.001)

In [ ]:
early_stopping_monitor = EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    verbose=1,
  )
hist = model.fit(X_train, y_train, epochs = 10, batch_size = 60, validation_data=(X_test,y_test), callbacks=[early_stopping_monitor])

In [ ]:
plt.plot(hist.history['val_loss'], color='b', label="validation loss")
plt.plot(hist.history['loss'], color='red', label="loss")
plt.title("Model Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['val_accuracy'], color='b', label="validation accuracy")
plt.plot(hist.history['accuracy'], color='red', label="accuracy")
plt.title("Model Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
y_pred = model1.predict(X_test)
y_pred = (y_pred > 0.5)
y_train_pred = model1.predict(X_train)
y_train_pred = (y_train_pred > 0.5)

In [ ]:
print('        Training Classification report for Bidirectional LSTM \n',classification_report(y_train,y_train_pred))
print('        Testing Classification Report for Bidirectional LSTM \n',classification_report(y_test,y_pred))



print('        Confusion Matrix for Testing Bidirectional LSTM \n')
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
print('        Confusion Matrix for Training Bidirectional LSTMr\n')
cm = confusion_matrix(y_train, y_train_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=[0,1])
disp.plot()
plt.show()

In [ ]:
indices = [i for i, (y1, y2) in enumerate(zip(y_test, y_pred)) if y1 == 0 and y2 == 1]

In [ ]:
print(indices)

In [ ]:
print(X_test[96])

In [ ]:
data.loc[97]

In [ ]:
df_f = data[data['fraudulent'] == 1]

In [ ]:
df_f['text'].head()